# infos

In [ ]:
# Version 1.3

# Modèle basé sur les 10 espèces les plus représentées du jeu de données.
# Images triées : Ok, choix possible
# Undersampling : Ok, choix possible
# Echantillonage : ok, choix possible

### Changelogs :

v1.3 :
- Utilisation de joblib pour sauvegarder l'historique d'entrainement au lieu de pickle
- Modification de la cellule de configuration pour la rendre plus interactive
- Certains noms de variables ont été modifiés pour une meilleure compéhrension
- Les fonctions perosnnalisées sont maintenant importées à partir d'une librairie construite 'fonctions' et 'config'


v1.2 :
- Ajout d'une fonction undersample pour équilibrer les observations
- La variable history (entrainement du modèle) est maintenant sauvegardée pour être exploitée dans d'autres notebooks
- Il est maintenant possible de définir en début de notebook :
    - Le csv utilisé (avec ou sans tri des images poubelles)
    - les données (full ou en echantillon avec choix de la taille)
    - L'undersampling (avec ou sans)

v1.1 :
- Nettoyage du code pour livraison projet
- Amélioration du modèle, entrainement sous format v1.1


v1.0 :
- Itération d'un modèle avec les images non triées et triées.
- Les modèles sont enregistrés sous format V1 pour tests ultérieurs

# Configuration Notebook

In [4]:
# Environnement Colab
#import sys
#sys.path.append('/content/drive/MyDrive/SAS/Jul23_bds_champignons/notebooks')

from config import config_env

work_env, chemin_images, data_file, data_name, data_size, pourcentage_echantillon, undersampling, model_name, history_path, img_dim, img_shape = config_env()


 Preciser le fichier de données à utiliser :
1. 10 classes, 64k images, non triées
2. 10 classes, 60k images, triées
Selection : 10 classes, 60k images, triées

 L'execution porte sur les données complètes ou un echantillon aléatoire des données ?:
1. Donnée complètes
2. Echantillon
Selection : Echantillon
Preciser la taille de l'echantillon, 0.1 correspondant à 10% des données
Taille echantillon : 10.0 %

 Les données doivent-elles équilibrées ? (un undersampling sera executé):
1. AVEC undersampling
2. SANS undersampling
Selection : Avec undersampling

 Preciser l'environnement de travail :
1. Ma machine
2. Google Colab
Environnement : Perso
Chemin des images: ../../images


In [5]:
1# Si l'environnement est Colab, ces cellules modifieront certains paramètres pour l'execution :
if work_env == '2':
  # Importer les images en format .zip
  #from google.colab import files
  #files.upload()

  # Monter le Drive
  from google.colab import drive
  drive.mount('/content/drive')

  import sys
  sys.path.append('/content/drive/MyDrive/SAS/Jul23_bds_champignons/notebooks')

  chemin_images = '/images/images/'
  data_file = '/content/drive/MyDrive/SAS/Jul23_bds_champignons/data/top10.csv'

else:
  print("L'environnement choisi n'est pas Google Colab, aucune modification apportée dans les chemins fichiers.")

L'environnement choisi n'est pas Google Colab, aucune modification apportée dans les chemins fichiers.


In [6]:
if work_env == '2':
  # Dezipper le fichier images dans Colab
  !unzip '/content/drive/MyDrive/SAS/images.zip' -d '/images'

# Librairies à charger

In [7]:
# Librairies générales servant dans le notebook
import pandas as pd

# Librairies appelées pour l'utilisation des fonctions définies dans la partie 'fonctions


# Librairies utilisées pour les callbacks
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import ModelCheckpoint
from timeit import default_timer as timer
from tensorflow.keras.callbacks import TerminateOnNaN


# Librairies utilisées pour créer les pipelines et le mµodèle
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense


# Librairies utilisées pour la création des jeux d'entrainement, de test et de validation
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


# Librairies utilisées pour des sauvegardes et chargements de variables
from joblib import dump

In [8]:
from fonctions import import_df, controle_presence_fichiers, create_tf_dataset, augment_img, undersampling_df

# Callbacks

### EarlyStopping

In [9]:
# earlystopping défini pour réduire le temps d'entrainement si l'accuracy n'évolue plus.
early_stopping = EarlyStopping(monitor = 'val_accuracy',
                               min_delta = 0.015,
                               patience = 5,
                               verbose = 1,
                               mode = 'auto',
                               restore_best_weights = True)

### Reduce LearningRate

In [10]:
# Reduction du learning Rate
reduceLR = ReduceLROnPlateau(monitor = 'val_loss',
                             min_delta = 0.01,
                             patience = 3,
                             factor = 0.15,
                             cooldown = 5,
                             verbose = 1)

### Checkpoint

In [11]:
# Enregistrement du modèle en cas d'amélioration lors de l'entrainement, ne conserve que la meilleure itération
checkpoint = ModelCheckpoint(filepath='../model/checkpoint_model', monitor='val_accuracy', save_best_only=True, verbose=1)

### Timer

In [12]:
# Suivi du temps d'entrainement
class TimingCallback(Callback):
    def __init__(self):
        super().__init__()
        self.logs = []

    def on_epoch_begin(self, epoch, logs=None):
        self.starttime = timer()

    def on_epoch_end(self, epoch, logs=None):
        endtime = timer()
        elapsed_time = endtime - self.starttime
        self.logs.append(elapsed_time)
        print(f"Epoch {epoch + 1} took {elapsed_time:.2f} seconds")

time_callback = TimingCallback()

### Terminate on NaN

In [13]:
# Termine l'entrainement en cas de NaN
TON = TerminateOnNaN()

# Construction modèle

### Import du modèle pré-entrainé

In [14]:
# import du modèle efficientNetv2 pré-entrainé depuis Tensorflow Hub, les couches de convolution sont gelées
efficientNetv2 = "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_b0/classification/2"
pre_trained_model = hub.KerasLayer(efficientNetv2, input_shape=img_shape, trainable=False)

### Definition des couches

In [15]:
# Couches de convolution :
reshape_layer = layers.Reshape((1, 1, 1000))  # Cette couche permet d'entrer la sortie du modèle pré-entrainé dans la couche GlobalAveragePooling2D
gap = GlobalAveragePooling2D()

# Couches de regularisation :
dropout1 = Dropout(0.3)
dropout2 = Dropout(0.2)

# Couches dense :
dense0 = Dense(1024, activation='relu')
dense1 = Dense(1024, activation='relu')
dense2 = Dense(512, activation='relu')
dense3 = Dense(256, activation='relu', kernel_regularizer=l2(0.01))
output = Dense(units = 10, activation='softmax') # Couche de sortie (10 classes)

### Construction du modèle

In [16]:
def build_model():

    model = tf.keras.Sequential()

    # Modèle pré-entrainé
    model.add(pre_trained_model)

    # Couches supplémentaires pour classification
    model.add(reshape_layer)
    model.add(gap)
    model.add(dense0)
    model.add(dropout1)
    model.add(dense1)
    model.add(dense2)
    model.add(dropout2)
    model.add(dense3)


    # Couche de sortie
    model.add(output)  # 10 classes de sortie

    return model

### Compilation du modèle

In [17]:
# Compiler le modèle
model = build_model()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [18]:
# Afficher un résumé du modèle
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1000)              7200312   
                                                                 
 reshape (Reshape)           (None, 1, 1, 1000)        0         
                                                                 
 global_average_pooling2d (  (None, 1000)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 1024)              1025024   
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                                 
 dense_1 (Dense)             (None, 1024)              1049600   
                                                        

# Pipeline Dataset

### Import des DataFrames

In [19]:
df, df_ech = import_df(chemin_images, data_file, pourcentage_echantillon)

# Le DataFrame qui sera load dans la pipeline tensorflow dépend du choix inital de taille des données :
if data_size == "1":
    training_data = df

elif data_size == "2":
    training_data = df_ech


# undersampling des classes : (si défini)
if undersampling == True:
  training_data = undersampling_df(training_data, col = 'label')

else:
  print("Pas d'undersampling programmé, les données sont déséquilibrées.")

Nombre d'images chargées pour df: 60481
Nb especes dans df: 10
Nombre d'images chargées pour df_ech: 6048
Nb especes dans df_ech: 10


d:\Projet champignons\Jul23_bds_champignons\notebooks\fonctions.py:24: FutureWarning: The default value of regex will change from True to False in a future version.
  df['image_url'] = df['image_url'].str.replace('../../images', chemin_images)


In [20]:
# Controle de la présence des fichiers images
controle_presence_fichiers(training_data, chemin_images)

# On supprime ensuite la colonne image_lien qui ne sert qu'à controler la présence des fichiers.
training_data.drop('image_lien', axis=1, inplace=True)


Tous les fichiers sont présents.


In [21]:
print(training_data.groupby('label').count())
training_data.head()

             image_url
label                 
Agaricales         387
Agaricus           387
Amanita            387
Cortinarius        387
Entoloma           387
Inocybe            387
Mycena             387
Polyporales        387
Psathyrella        387
Russula            387


,label,image_url
0,Agaricales,../../images/461645.jpg
1,Agaricales,../../images/334685.jpg
2,Agaricales,../../images/507041.jpg
3,Agaricales,../../images/411553.jpg
4,Agaricales,../../images/561057.jpg


### Construction des jeux de données (train, test et validation)

In [22]:
data = training_data.drop('label', axis=1)
target = training_data['label']

 # Encodage de la variable 'label'
s = LabelEncoder()
target = s.fit_transform(target)

# On construit le jeu d'entrainnement. X_temp et y_temps servent pour la construction des jeux de test et validation
X_train, X_temp, y_train, y_temp = train_test_split(data, target, test_size=0.20, random_state=10)

# On split les temp en 50% pour test, 50% pour validation
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=10)

### Construction des dataset Tensorflow

In [23]:
image_paths = X_train.image_url.tolist()
labels = y_train.tolist()
batch_size = 64
augment = True

# Les datasets sont créés à partir de la fonction create_tf_dataset définie dans la partie 'Fonctions'
ds_train = create_tf_dataset(image_path = X_train.image_url, img_dim = img_dim, labels = y_train, batch_size = batch_size, augment = augment)
ds_val = create_tf_dataset(image_path = X_val.image_url, img_dim = img_dim, labels = y_val, batch_size = batch_size, augment = augment)


# Entrainement du modèle

### Methode .fit

In [27]:
history = model.fit(ds_train,
                    validation_data = ds_val,
                    epochs=50,
                    callbacks = [early_stopping, reduceLR, checkpoint, time_callback, TON],
                    verbose=True)

Epoch 1/2


49/49 [==============================] - ETA: 0s - loss: 4.4173 - accuracy: 0.2988
Epoch 1: val_accuracy improved from -inf to 0.37726, saving model to ../model\checkpoint_model
INFO:tensorflow:Assets written to: ../model\checkpoint_model\assets


INFO:tensorflow:Assets written to: ../model\checkpoint_model\assets


Epoch 1 took 56.52 seconds
49/49 [==============================] - 57s 1s/step - loss: 4.4173 - accuracy: 0.2988 - val_loss: 3.1849 - val_accuracy: 0.3773 - lr: 0.0010
Epoch 2/2
49/49 [==============================] - ETA: 0s - loss: 2.6718 - accuracy: 0.4816
Epoch 2: val_accuracy improved from 0.37726 to 0.51421, saving model to ../model\checkpoint_model
INFO:tensorflow:Assets written to: ../model\checkpoint_model\assets


INFO:tensorflow:Assets written to: ../model\checkpoint_model\assets


Epoch 2 took 53.77 seconds
49/49 [==============================] - 54s 1s/step - loss: 2.6718 - accuracy: 0.4816 - val_loss: 2.2167 - val_accuracy: 0.5142 - lr: 0.0010


# Sauvegarde du modèle

In [37]:
save_history = 'history_' + model_name


if work_env == '2':
    model_name = '/content/drive/MyDrive/SAS/model/' + model_name
    history_path = '/content/drive/MyDrive/SAS/history/'+ save_history + '.joblib'

else:
    model_name = '../model/' + model_name
    history_path = '../history/'+ save_history + '.joblib'


model.save(model_name)


with open(history_path, 'wb') as file:
  dump(history, file)         #joblib